In [1]:
import sqlalchemy as db
import pandas as pd

In [2]:
engine = db.create_engine("sqlite:///mta_data.db")

In [3]:
df = pd.read_sql("SELECT * FROM mta_data;", engine)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17191756 entries, 0 to 17191755
Data columns (total 11 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   C/A       object
 1   UNIT      object
 2   SCP       object
 3   STATION   object
 4   LINENAME  object
 5   DIVISION  object
 6   DATE      object
 7   TIME      object
 8   DESC      object
 9   ENTRIES   int64 
 10  EXITS     int64 
dtypes: int64(2), object(9)
memory usage: 1.4+ GB


In [5]:
crazy_stations = ['GRD CNTRL-42 ST', 'WALL ST', 'HOWARD BCH JFK', '161/YANKEE STAD', 'CONEY IS-STILLW',
                  '116 ST-COLUMBIA', 'FORDHAM RD', 'BRONX PARK EAST', 'BEACH 67 ST', 'JAMAICA VAN WK',
                  'BROOKLYN BRIDGE', 'LAFAYETTE AV', 'STEINWAY ST', '14 ST']

In [6]:
crazy_df = df[df.STATION.isin(crazy_stations)]

In [7]:
crazy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1127193 entries, 30651 to 17161750
Data columns (total 11 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   C/A       1127193 non-null  object
 1   UNIT      1127193 non-null  object
 2   SCP       1127193 non-null  object
 3   STATION   1127193 non-null  object
 4   LINENAME  1127193 non-null  object
 5   DIVISION  1127193 non-null  object
 6   DATE      1127193 non-null  object
 7   TIME      1127193 non-null  object
 8   DESC      1127193 non-null  object
 9   ENTRIES   1127193 non-null  int64 
 10  EXITS     1127193 non-null  int64 
dtypes: int64(2), object(9)
memory usage: 103.2+ MB


In [8]:
crazy_df.loc[:, "DATE_FORMATTED"] = pd.to_datetime(crazy_df.loc[:, "DATE"])

/Users/william.bell/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [9]:
crazy_df = crazy_df[crazy_df.DATE_FORMATTED.dt.year.isin([2019,2021])]
crazy_df.sample(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_FORMATTED
11214982,R240,R047,00-00-01,GRD CNTRL-42 ST,4567S,IRT,07/26/2019,21:00:00,REGULAR,1074589,1216758,2019-07-26
3516967,R241A,R048,00-00-00,GRD CNTRL-42 ST,4567S,IRT,06/05/2021,21:00:00,REGULAR,1660506,2157990,2021-06-05
13456460,R112,R027,02-00-01,WALL ST,23,IRT,05/08/2019,00:00:00,REGULAR,8114247,9073422,2019-05-08
14311637,R262B,R195,05-00-04,161/YANKEE STAD,4BD,IRT,04/12/2019,12:22:00,REGULAR,695473,625783,2019-04-12
8439176,N110,R283,00-03-00,LAFAYETTE AV,C,IND,10/19/2019,09:00:00,REGULAR,487882,1668264,2019-10-19


In [10]:
def get_daily_counts(row, max_counter, turnstile_direction):
    count_name = turnstile_direction
    prev_count_name = f"PREV_{turnstile_direction}"
    counter = row[count_name] - row[prev_count_name]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0?
        print(row[count_name], row[prev_count_name])
        counter = min(row[count_name], row[prev_count_name])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

In [11]:
def turnstiles_daily(data, turnstile_direction):
    turnstiles_df = (data.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_FORMATTED"], as_index=False)[turnstile_direction].max())
    turnstiles_df[["PREV_DATE", f"PREV_{turnstile_direction}"]] = turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION"])[["DATE_FORMATTED", turnstile_direction]].shift(1)
    turnstiles_df.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
    turnstiles_df[f"DAILY_{turnstile_direction}"] = turnstiles_df.apply(get_daily_counts, axis=1, max_counter=500000,turnstile_direction=turnstile_direction)
    turnstiles_df = turnstiles_df[turnstiles_df["DATE_FORMATTED"] != pd.Timestamp(2021, 1, 1)]
    return turnstiles_df

In [12]:
turnstile_entries = turnstiles_daily(crazy_df, "ENTRIES")
turnstile_entries.sample(20)

101 2651127.0
104 17275602.0
91 863563.0
1588298 988219.0
2726 6527930.0
167226 13768271.0
8855170 8099782.0
22 2129136933.0
21 13593868.0
27 7808626.0
21 2129136817.0
28 13594033.0
21 7809012.0
9855 2369738.0
66 500623.0
1962917910 9137598.0
8408479 1740444.0
2564432 483386.0
1627455719 6233261.0
23157717 1627480864.0
1626 557914846.0
721441460 2880424.0
916 5405537.0
19295409 18704160.0
687970551 19471253.0
80119 1347009.0
8989 751823.0
263461 1252108.0
484803 1063922.0
2624 4676995.0
1963 621485465.0
819 6214051.0
41042 1003676.0
15970 5862828.0
35622 5000575.0
14526 2645740.0
2868212 11143833.0
191 2811568.0
691 9646557.0
9789 1598582.0
72877 3952604.0
1099 4001182.0
956 5469421.0
1881 5144420.0
1629 5959606.0
59838 2202393.0
91648 12958832.0
14415 1836614.0
11144978 10616824.0
1338 1427069.0
68448 11726783.0
25771321 25257660.0
939 1372039.0
592 749149.0
1569915 831516.0
22 1571404.0
629 597948.0
524954 167809675.0
459330 51009835.0
50362839 251769.0
1466 519056.0
29510 10939334.0

,C/A,UNIT,SCP,STATION,DATE_FORMATTED,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
49937,N218,R112,01-05-01,FORDHAM RD,2019-09-06,0,2019-09-05,0.0,0.0
77485,R111,R027,00-00-00,WALL ST,2019-08-30,2108579,2019-08-29,2106927.0,1652.0
120207,R210A,R044,03-03-00,BROOKLYN BRIDGE,2019-06-28,4777497,2019-06-27,4776680.0,817.0
45746,N217,R112,00-00-00,FORDHAM RD,2019-03-02,4030523,2019-03-01,4029348.0,1175.0
1786,G001,R151,00-00-03,CONEY IS-STILLW,2019-03-12,7045335,2019-03-11,7044946.0,389.0
24093,N110,R283,00-00-01,LAFAYETTE AV,2019-02-17,458121,2019-02-16,458070.0,51.0
79295,R111,R027,00-00-03,WALL ST,2021-02-07,5589709,2021-02-06,5589679.0,30.0
179754,R289,R119,00-03-01,FORDHAM RD,2019-01-10,17900066,2019-01-09,17897266.0,2800.0
9254,G009,R151,02-00-04,CONEY IS-STILLW,2019-05-03,7515549,2019-05-02,7513599.0,1950.0
61171,N511,R163,03-06-00,14 ST,2019-02-12,2677733,2019-02-11,2677600.0,133.0


In [19]:
pd.set_option('display.max_rows', 500)
turnstile_entries.sort_values(by="DAILY_ENTRIES", ascending=False).head(100)

,C/A,UNIT,SCP,STATION,DATE_FORMATTED,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
132859,R236,R045,00-2E-00,GRD CNTRL-42 ST,2019-07-27,459479,2019-07-16,6.0,459473.0
109870,R204,R043,02-06-00,WALL ST,2019-03-02,459330,2019-03-01,51009835.0,459330.0
132329,R236,R045,00-06-00,GRD CNTRL-42 ST,2019-08-10,1246,2019-08-09,460155.0,458909.0
118989,R210,R044,00-06-00,BROOKLYN BRIDGE,2019-04-14,458781,2019-04-13,120551519.0,458781.0
118549,R210,R044,00-05-01,BROOKLYN BRIDGE,2019-08-24,458755,2019-08-22,0.0,458755.0
103713,R203A,R043,01-05-01,WALL ST,2019-07-04,458752,2019-07-03,0.0,458752.0
181012,R289,R119,00-05-01,FORDHAM RD,2019-05-04,458752,2019-05-03,100663296.0,458752.0
132313,R236,R045,00-06-00,GRD CNTRL-42 ST,2019-07-25,459338,2019-07-17,941.0,458397.0
5376,G001,R151,00-05-00,CONEY IS-STILLW,2019-08-20,458754,2019-08-19,3774.0,454980.0
110090,R204,R043,02-06-00,WALL ST,2019-10-13,250,2019-10-12,431482.0,431232.0


In [20]:
turnstiles_daily(crazy_df, "EXITS").sort_values(by="DAILY_EXITS", ascending=False).head(100)

137 2192493.0
161 1075170.0
11508703 10994314.0
1394960 813409.0
4587 8480804.0
1000553 456374.0
201430 16003648.0
16 1155490186.0
14 15837645.0
15 4262196.0
17 1155489758.0
12 15838154.0
14 4262925.0
11215 1548577.0
59 805938.0
318790192 1305662.0
81 625283.0
1627545404 6176424.0
56711037 1627593487.0
831 186124516.0
1895802233 9395822.0
510 1852012.0
1174467683 4611970.0
143984 1660552.0
7350508 6562329.0
4478 7279490.0
972 369533236.0
164 767349.0
11183 2868410.0
82479 6585424.0
30414 4912792.0
87672 13802238.0
426 3505140.0
526 8377614.0
32322 1471054.0
11889464 11121772.0
49700 1970485.0
843 3084910.0
1338 3494920.0
101 4414526.0
1446 1386907.0
61940 2429376.0
36008 4634180.0
11595 1516070.0
11911612 11235879.0
775 736048.0
2020 5739414.0
9577 4293087.0
2885 1256717.0
38911 1473988.0
672 18410825.0
604001837 2369108.0
19832 2655224.0
17 900047.0
10441305 5072366.0
5075449 10441305.0
38467 1885174.0
883 1009858.0
886336073 202280.0
5911484 5331393.0
4983732 789409.0
57 4983732.0
20

,C/A,UNIT,SCP,STATION,DATE_FORMATTED,EXITS,PREV_DATE,PREV_EXITS,DAILY_EXITS
98003,R203,R043,00-00-01,WALL ST,2019-07-03,141,2019-07-01,480710.0,480569.0
115487,R210,R044,00-03-02,BROOKLYN BRIDGE,2019-02-01,1493,2019-01-31,436617.0,435124.0
132321,R236,R045,00-06-00,GRD CNTRL-42 ST,2019-08-02,327685,2019-08-01,442.0,327243.0
12070,JFK01,R535,00-00-02,HOWARD BCH JFK,2019-03-25,744,2019-03-24,276743.0,275999.0
132318,R236,R045,00-06-00,GRD CNTRL-42 ST,2019-07-30,262318,2019-07-29,117.0,262201.0
132313,R236,R045,00-06-00,GRD CNTRL-42 ST,2019-07-25,262231,2019-07-17,71.0,262160.0
132322,R236,R045,00-06-00,GRD CNTRL-42 ST,2019-08-03,65541,2019-08-02,327685.0,262144.0
132314,R236,R045,00-06-00,GRD CNTRL-42 ST,2019-07-26,149,2019-07-25,262231.0,262082.0
132319,R236,R045,00-06-00,GRD CNTRL-42 ST,2019-07-31,388,2019-07-30,262318.0,261930.0
130602,R236,R045,00-03-01,GRD CNTRL-42 ST,2019-07-23,886336073,2019-07-22,202280.0,202280.0
